In [3]:
# Ce script crée une carte du monde montrant la consommation d'alcool par pays avec un code couleur
import pandas as pd
import plotly.express as px
import pycountry
from plotly.subplots import make_subplots
import plotly.graph_objects as go

donnees = pd.read_csv('donnée.csv')
donnees = donnees[donnees['Year'] == donnees['Year'].max()]
donnees = donnees.rename(columns={
    'Entity': 'Pays',
    'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)': 'Consommation Alcool'
})

donnees = donnees.dropna(subset=['Consommation Alcool'])

def obtenir_code_iso(nom_pays):
    try:
        return pycountry.countries.lookup(nom_pays).alpha_3
    except:
        return None

donnees['Code ISO'] = donnees['Pays'].apply(obtenir_code_iso)
donnees = donnees.dropna(subset=['Code ISO'])

def ajouter_commentaire(consommation):
    if consommation > 15:
        return "🍻 Champion du monde !"
    elif consommation > 12:
        return "🥂 Très festif"
    elif consommation > 9:
        return "🍷 Amateur éclairé"
    elif consommation > 6:
        return "🍺 Modéré"
    elif consommation > 3:
        return "🍹 Occasionnel"
    else:
        return "🧃 Sobre ou abstinent"

donnees['Commentaire'] = donnees['Consommation Alcool'].apply(ajouter_commentaire)

carte = px.choropleth(
    donnees,
    locations='Code ISO',
    color='Consommation Alcool',
    hover_name='Pays',
    hover_data={
        'Consommation Alcool': ':.2f',
        'Commentaire': True,
        'Code ISO': False
    },
    color_continuous_scale='Reds',
    title='Consommation annuelle d\'alcool par habitant (litres d\'alcool pur)'
)

carte.update_layout(
    geo=dict(showframe=False, showcoastlines=False),
    coloraxis_colorbar=dict(title='Litres par an')
)

carte.show()

In [4]:
# Ce script analyse la consommation d'alcool par continent et crée un graphique à barres horizontales
continent_mapping = {
    'Europe': ['FRA', 'DEU', 'GBR', 'ITA', 'ESP', 'POL', 'ROU', 'NLD', 'BEL', 'GRC'],
    'Asie': ['CHN', 'JPN', 'KOR', 'IND', 'IDN', 'VNM', 'THA', 'MYS', 'PHL', 'SGP'],
    'Amérique': ['USA', 'CAN', 'MEX', 'BRA', 'ARG', 'COL', 'CHL', 'PER', 'VEN', 'URY'],
    'Afrique': ['ZAF', 'EGY', 'MAR', 'TUN', 'DZA', 'NGA', 'KEN', 'ETH', 'GHA', 'CIV'],
    'Océanie': ['AUS', 'NZL', 'PNG', 'FJI', 'NCL', 'VUT', 'SLB', 'WSM', 'TON']
}

def attribuer_continent(code_iso):
    for continent, pays in continent_mapping.items():
        if code_iso in pays:
            return continent
    return "Autre"

donnees['Continent'] = donnees['Code ISO'].apply(attribuer_continent)
moyennes_continent = donnees.groupby('Continent')['Consommation Alcool'].mean().sort_values(ascending=True)

fig_continents = px.bar(
    moyennes_continent,
    orientation='h',
    title='Consommation moyenne d\'alcool par continent',
    labels={'value': 'Litres d\'alcool pur par an', 'Continent': 'Continent'},
    color_discrete_sequence=['#ff6b6b']
)

fig_continents.update_layout(
    showlegend=False,
    xaxis_title="Litres d'alcool pur par an",
    yaxis_title="Continent"
)

fig_continents.show()

In [5]:
# Ce script identifie et visualise le top 10 des pays consommant le plus d'alcool
top_10_pays = donnees.nlargest(10, 'Consommation Alcool')

fig_top10 = px.bar(
    top_10_pays,
    x='Pays',
    y='Consommation Alcool',
    title='Top 10 des pays avec la plus forte consommation d\'alcool',
    labels={
        'Consommation Alcool': 'Litres d\'alcool pur par an',
        'Pays': 'Pays'
    },
    text='Commentaire',
    color='Consommation Alcool',
    color_continuous_scale='Reds'
)

fig_top10.update_traces(textposition='outside')
fig_top10.update_layout(
    xaxis_tickangle=45,
    showlegend=False
)

fig_top10.show()

In [8]:
# Ce script analyse l'évolution de la consommation d'alcool pour les 5 plus gros consommateurs
donnees_complete = pd.read_csv('donnée.csv')
top_5_pays = donnees['Pays'].head(5).tolist()
donnees_evolution = donnees_complete[donnees_complete['Entity'].isin(top_5_pays)]

fig_evolution = px.line(
    donnees_evolution,
    x='Year',
    y='Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
    color='Entity',
    title='Évolution de la consommation d\'alcool des 5 plus gros consommateurs (2000-2020)',
    labels={
        'Year': 'Année',
        'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)': 'Litres d\'alcool pur par an',
        'Entity': 'Pays'
    }
)

fig_evolution.update_layout(
    xaxis_title="Année",
    yaxis_title="Litres d'alcool pur par an",
    legend_title="Pays"
)

fig_evolution.show()

In [ ]:
# Calcul de la moyenne mondiale par année
moyenne_mondiale = donnees_complete.groupby('Année')[
    'Consommation totale d\'alcool par habitant (litres d\'alcool pur, estimations projetées, 15 ans et plus)'
].mean().reset_index()

# Calcul de la médiane mondiale par année
mediane_mondiale = donnees_complete.groupby('Année')[
    'Consommation totale d\'alcool par habitant (litres d\'alcool pur, estimations projetées, 15 ans et plus)'
].median().reset_index()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=moyenne_mondiale['Année'],
        y=moyenne_mondiale['Consommation totale d\'alcool par habitant (litres d\'alcool pur, estimations projetées, 15 ans et plus)'],
        mode='lines',
        name='Moyenne mondiale',
        line=dict(color='red', width=3)
    )
)

fig.add_trace(
    go.Scatter(
        x=mediane_mondiale['Année'],
        y=mediane_mondiale['Consommation totale d\'alcool par habitant (litres d\'alcool pur, estimations projetées, 15 ans et plus)'],
        mode='lines',
        name='Médiane mondiale',
        line=dict(color='blue', width=3, dash='dash')
    )
)

fig.update_layout(
    title="Évolution de la consommation mondiale d'alcool (2000-2020)",
    xaxis_title="Année",
    yaxis_title="Litres d'alcool pur par an",
    showlegend=True,
    width=1000,
    height=600,
    template='simple_white'
)

fig.show()

In [ ]:
# Ce script crée un tableau de mini-graphiques montrant l'évolution pour chaque pays
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pays_uniques = donnees_complete['Pays'].unique()
n_pays = len(pays_uniques)
n_colonnes = 4
n_lignes = (n_pays - 1) // n_colonnes + 1

fig = make_subplots(rows=n_lignes, cols=n_colonnes, subplot_titles=pays_uniques)

for i, pays in enumerate(pays_uniques):
    ligne = i // n_colonnes + 1
    colonne = i % n_colonnes + 1
    
    donnees_pays = donnees_complete[donnees_complete['Pays'] == pays]
    
    fig.add_trace(
        go.Scatter(
            x=donnees_pays['Année'], 
            y=donnees_pays['Consommation totale d\'alcool par habitant (litres d\'alcool pur, estimations projetées, 15 ans et plus)'],
            name=pays,
            showlegend=False
        ),
        row=ligne, col=colonne
    )

fig.update_layout(
    height=300 * n_lignes,
    title_text="Évolution de la consommation d'alcool par pays (2000-2020)",
    showlegend=False
)

fig.update_xaxes(title_text="Année")
fig.update_yaxes(title_text="Litres par an")

fig.show()
